This file tests the deduplication code from EleutherAI's `janitor.py` file on small section(s) of Dolma to estimate how long full deduplication would take.

In [1]:
# (Not working) For parallelization on GPUs
import numba

SystemError: initialization of _internal failed without raising an exception

In [2]:
from numba import jit, cuda
import pyarrow.parquet as pq
from pathlib import Path
import sys
import datetime

harness_dir = str(Path("__file__").resolve().parents[3] / "lm-evaluation-harness")
sys.path.append(harness_dir)

# To run this with C++ on Linux:
# At lm-evaluation-harness/scripts/clean_training_data,
# run c++ -O3 -Wall -shared -std=c++11 -fPIC $(python3 -m pybind11 --includes) janitor_util.cpp -o janitor_util$(python3-config --extension-suffix)
# Rename the resulting .so file to janitor_ util.so
# Tell Python the location of janitor_util.so when it looks for janitor_util
sys.path.append(harness_dir + "/scripts/clean_training_data")
from lm_eval.decontamination.janitor import Janitor

SystemError: initialization of _internal failed without raising an exception

In [2]:
with open("./arithmetic.txt", "r") as file:
    arithmetic = file.read()

In [7]:
# Decontaminate 8 files of c4 (no parallelization)
total_decontamination_time = datetime.timedelta(hours=0)
janitor = Janitor(delete_chars="")
janitor.register_contaminant(arithmetic)

for file_num in range(5, 13):
    arrow_path = (
        f"/data/tir/projects/tir7/user_data/mchen5/dolma_100B/c4/part_{file_num}.arrow"
    )
    data = pq.read_table(arrow_path)
    data_string = "".join(data.column("text").to_pandas())
    print(f"Loaded c4/part_{file_num}.arrow")

    pre_decontaminate = datetime.datetime.now()
    # Test decontaminating arithmetic against c4 part {file_num}
    result = janitor.clean_python(data_string)

    print(
        f"Decontaminated c4 part {file_num} of arithmetic in "
        + str(datetime.datetime.now() - pre_decontaminate)
    )

    total_decontamination_time += datetime.datetime.now() - pre_decontaminate

Loaded c4/part_5.arrow


For estimating runtimes:
| Folder | # of arrow files |
|--------|-------|
| c4 | 4213499 |
| common-crawl | 510983 |
| gutenberg-books | 1178 |
| peS2o | 20803 |
| stack-code | 103818 |
| wiki-en-simple | 2785999 |
| dolma_100B (total) | 7636280 |


Testing results:
- Deduplicating full part 1 of c4 against arithmetic (C++):
    - 512G RAM (43 G used), 1 GPU, 4 CPUs (169% efficiency) - 15 mins 40 sec
- Deduplicating 1/10 of part 1 of c4 against arithmetic (Python):
    - 512G RAM (43 G used), 1 GPU, 4 CPUs (169% efficiency) - 1 min 35 sec
- Deduplicating full part 2 of c4 against arithemtic (C++):
    - 512G RAM (26 G used), 1 GPU, 16 CPUs (14% efficiency) - 16 min 16 sec
- Deduplicating parts 5 - 12 of c4 against arithmetic (C++):
    - 512G RAM (43 G used), 1 GPU, 16 CPUs (66% efficiency) - 140 min
        - Average 17.5 min per part

To do:
- Multithread(?) by splitting data into chunks and deduplicating each chunk in parallel